## Deploy Text Embedding Model: Sentence transformer(embedder) all-MiniLM-L6-v2

In [6]:
!pip install -qU sentence-transformers


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


#### Imports

In [10]:
from sagemaker.predictor import Predictor
from sagemaker import get_execution_role
from sagemaker.model import Model
from sagemaker import script_uris
from sagemaker import image_uris 
from sagemaker import model_uris
import sagemaker
import boto3
import time
import json

#### Setup essentials 

##### Setup config params

In [11]:
model_id = "huggingface-textembedding-all-MiniLM-L6-v2"  
model_version = "*"
instance_type = "ml.g5.xlarge"
instance_count = 1
image_scope = "inference"
model_data_download_timeout = 3600  # in seconds
container_startup_health_check_timeout = 3600
content_type = "application/json"

# Set up roles and clients 
client = boto3.client("sagemaker-runtime")
role = get_execution_role()

In [13]:
unix_time = int(time.time())
endpoint_name = f"{model_id}-{unix_time}"
print(f"Endpoint name: {endpoint_name}")

Endpoint name: huggingface-textembedding-all-MiniLM-L6-v2-1686825932


#### Retrieve image and model URIs

In [16]:
image_uri = image_uris.retrieve(framework=None, 
                                region=None,
                                image_scope=image_scope, 
                                model_id=model_id, 
                                model_version=model_version, 
                                instance_type=instance_type)
print(f"Inference image URI: {image_uri}")

Inference image URI: 763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-inference:1.10.2-transformers4.17.0-gpu-py38-cu113-ubuntu20.04


In [17]:
model_uri = model_uris.retrieve(model_id=model_id, 
                                model_version=model_version, 
                                model_scope=image_scope)
print(f"Model URI: {model_uri}")

Model URI: s3://jumpstart-cache-prod-us-east-1/huggingface-infer/prepack/v1.0.0/infer-prepack-huggingface-textembedding-all-MiniLM-L6-v2.tar.gz


In [11]:
# env = {
#     'SAGEMAKER_MODEL_SERVER_TIMEOUT': str(3600),
#     'MODEL_CACHE_ROOT': '/opt/ml/model', 
#     'SAGEMAKER_ENV': '1',
#     'SAGEMAKER_SUBMIT_DIRECTORY': '/opt/ml/model/code/',
#     'SAGEMAKER_PROGRAM': 'inference.py',
#     'SAGEMAKER_MODEL_SERVER_WORKERS': '1', 
#     'TS_DEFAULT_WORKERS_PER_MODEL': '1', 
# }

#### Create SageMaker Model

In [21]:
model = Model(image_uri=image_uri, 
              model_data=model_uri, 
              role=role, 
              predictor_cls=Predictor, 
              name=endpoint_name)

#### Deploy text embedding model as SageMaker endpoint for real-time synchronous inference

In [22]:
%%time

model.deploy(initial_instance_count=instance_count, 
                 instance_type=instance_type, 
                 endpoint_name=endpoint_name, 
                 model_data_download_timeout=model_data_download_timeout, 
                 container_startup_health_check_timeout=container_startup_health_check_timeout)

-------!CPU times: user 81.7 ms, sys: 7.68 ms, total: 89.4 ms
Wall time: 4min 3s
